In [ ]:
// Use TestContainers to start a small Docker Postgres database
@file:DependsOn("org.testcontainers:junit-jupiter:1.21.3")
@file:DependsOn("org.testcontainers:postgresql:1.20.0")

In [ ]:
// Start container and import initial database
import org.testcontainers.containers.PostgreSQLContainer
import org.testcontainers.utility.DockerImageName
import org.testcontainers.utility.MountableFile

val postgres = PostgreSQLContainer(DockerImageName.parse("postgres:16-alpine"))
    .withCopyToContainer(
        MountableFile.forHostPath("../src/test/resources/init_postgres.sql"),
        "/docker-entrypoint-initdb.d/init.sql"
    )
postgres.start()
DISPLAY("Is running: ${postgres.isRunning}")
DISPLAY("URL: ${postgres.getJdbcUrl()}")

In [ ]:
// Create a DataSource for the Postgres instance
val dataSrc = createDataSrc(
    jdbcUrl = postgres.jdbcUrl,
    username = "test",
    password = "test"
)
dataSrc

In [ ]:
// Include DataFrame so we can easily import data
%use dataframe

In [ ]:
dataSrc.connection.use {
    DataFrame.readSqlTable(it, "users")
}

In [ ]:
// Remember to stop the container after use to prevent it from leaking
postgres.stop()